# Correlation of PPE Demand in USA With Covid19 Cases

In [ ]:
# import data
import json, requests
import os, sys, time
import os.path as path
import configparser

# computing
import pandas as pd
import numpy as np

# Import geopandas package
import plotly.express as px

### Add python path and add local packages

In [ ]:
# Add getusppe_viz/src to python path (two up)
two_up = path.abspath(path.join(os.getcwd(),"../.."))
src_dir = os.path.join(two_up,'src')
if src_dir not in sys.path:
    sys.path.append(src_dir)

# Import local libraries
from geocode import geocoder
from mapping import choropleth_mapbox_usa_plot
from data_download import download_findthemasks_data, \
    download_nytimes_data, \
    download_hospital_data
from data_process import add_fips_county_info, \
    requests_per_county, \
    download_county_geojson_and_merge_df, \
    merge_covid_ppe_df, \
    process_hospital_data, \
    merge_covid_ppe_hosp_df, \
    calculate_covid_per_bed_available, \
    find_counties_with_covid19_and_no_ppe_request, \
    add_all_ppe_requests_to_merged_df

### Configs

In [ ]:
config = configparser.ConfigParser()
config.read(os.path.join(src_dir,'config.ini'))

ny_times_covid_date = config['viz']['ny_times_covid_date']
findthemasks_url = config['viz']['findthemasks_url']
request_headers = eval(config['viz']['request_headers'])
county_fips_download_url = config['viz']['county_fips_download_url']
geojson_url = config['viz']['geojson_url']
ny_times_county_data_url = config['viz']['ny_times_county_data_url']
hospital_download_url = config['viz']['hospital_download_url']

## Download find the mask data and convert to pandas
- Taken from find the mask [web visualization](https://findthemasks.com/give.html) 
- [Data updated every 5 mins here](findthemasks.com/data.json) - The data visulized here is from 3/25 at 10PM PST

In [ ]:
mask_df = download_findthemasks_data(
    url = findthemasks_url, request_headers = request_headers)
mask_df.head(2)

### Save out temporary list of data from specific states

In [ ]:
states = ['AZ','AR','UT','ID','CO']
states_of_interest = mask_df[mask_df.State.isin(states)]
path = 'Temp_data' + time.strftime("%Y%m%d") + '.csv'
states_of_interest.to_csv (path, index = False, header=True)

### Create geocoder class to find fips and county information by lat/long

In [ ]:
geocoder = geocoder(county_fips_download_url)

### Search and add the FIPS code to each row - WILL TAKE SEVERAL MINS

In [ ]:
mask_df = add_fips_county_info(mask_df, geocoder)

In [ ]:
timestr = time.strftime("%Y%m%d")
path = 'mask_df_data_by_county_' + timestr + '.csv'
mask_df.to_csv (path, index = False, header=True)

### Sum amount of requests per county

In [ ]:
mask_df_counties = requests_per_county(mask_df, write_out_csv = True)
mask_df_counties.head(5)

### Download county geo information

In [ ]:
counties, merged_df = download_county_geojson_and_merge_df(geojson_url, mask_df_counties)
merged_df.head(2)

### Add list of all ppe requests per county

In [ ]:
merged_df = add_all_ppe_requests_to_merged_df(mask_df,merged_df)

In [ ]:
# TODO : Add all requests info as clickable info on the interactive maps
# How to pull array of dicts from 'all_ppe_requests' category
''' 
all_ppe_locations_array= eval(str(merged_df.loc[
    merged_df['fips'] == '01073', 'all_ppe_requests'].values[0]))
print (all_ppe_locations_array)
merged_df['ppe_text'] = 'PPE Requests: ' + merged_df['PPE_requests'].astype(int).astype(str) + '<br>'+ \
        merged_df['county'].astype(str) + ', ' + merged_df['STATE'].astype(str) + \
        eval(merged_df['all_ppe_requests'].values
'''

### Map of PPE Requests

In [ ]:
choropleth_mapbox_usa_plot(
    counties = counties,
    locations = merged_df.fips,
    z = merged_df.PPE_requests,
    text = merged_df.ppe_text,
    #colorscale = ["#fdfcef", "#c7e9b4", "#6ab7a6","#41b6c4","#2c7fb8","#253494"],
    colorscale = ["#fdfcef","#c7e9b4","#D2FBFF","#36A2B9","#004469"],
    zmin = 0,
    zmax=5,
    title = ('PPE Requests By County - %s - (Hover for breakdown)' % time.strftime("%Y%m%d")),
    colorbar_title = '> PPE Requests',
    html_filename = 'PPE_Requests_By_County.html')

## Download COVID19 data and convert to pandas

In [ ]:
covid_df = download_nytimes_data(
    ny_times_county_data_url, ny_times_covid_date, write_out_csv = True)
covid_df.head(5)

### Merge the covid and Nytimes data

In [ ]:
merged_covid_ppe_df=merge_covid_ppe_df(covid_df,merged_df) 
merged_covid_ppe_df.head(2)

### Mapping covid cases 

In [ ]:
choropleth_mapbox_usa_plot(
    counties = counties,
    locations = merged_covid_ppe_df.fips,
    z = merged_covid_ppe_df.cases,
    text = merged_covid_ppe_df.covid_text,
    colorscale = ["#fdfcef","#ffda55","#FFC831","#fc7555","#e96e81",],
    zmin = 0,
    zmax=100,
    title = ('COVID19 Cases Per County - %s - (Hover for breakdown)' % ny_times_covid_date),
    html_filename = ('COVID19_Cases_Per_County_%s.html' % ny_times_covid_date),
    colorbar_title = '> COVID19 Cases',
)

## Hospital bed visualization by county 

In [ ]:
hospital_df = download_hospital_data(hospital_download_url, write_out_csv = True)
hospital_df.head(3)

In [ ]:
hospital_df_counties = process_hospital_data(hospital_df, write_out_csv = True)
hospital_df_counties.head(2)

In [ ]:
merged_covid_ppe_hosp_df=merge_covid_ppe_hosp_df(hospital_df_counties,merged_covid_ppe_df) 
merged_covid_ppe_hosp_df.head(2)

In [ ]:
# Hospital bed plotting
choropleth_mapbox_usa_plot(
    counties = counties,
    locations = merged_covid_ppe_hosp_df.fips,
    z = merged_covid_ppe_hosp_df.BEDS,
    text = merged_covid_ppe_hosp_df.hosp_text,
    colorscale = ["#fdfcef","#c7e9b4","#D2FBFF","#36A2B9","#004469"],
    zmin = 0,
    zmax=500,
    title = ('Hospital beds per county - %s - (Hover for breakdown)' % time.strftime("%Y%m%d")),
    html_filename = ('Hospital_beds_per_county_%s.html' % time.strftime("%Y%m%d")),
    colorbar_title = '> Hospital Beds'
    )

### Covid cases per bed available

In [ ]:
merged_covid_ppe_hosp_df = calculate_covid_per_bed_available(merged_covid_ppe_hosp_df)
merged_covid_ppe_hosp_df.head(2)

In [ ]:
# Map hazard ratio
choropleth_mapbox_usa_plot(
    counties = counties,
    locations = merged_covid_ppe_hosp_df.fips,
    z = merged_covid_ppe_hosp_df.Covid_cases_per_bed,
    text = merged_covid_ppe_hosp_df.hosp_text,
    colorscale = ["#fdfcef","#ffda55","#FFC831","#fc7555","#e96e81",],
    zmin = 0,
    zmax=1,
    title = ('Hazard Ratio: Covid19 Cases, Per Bed, Per County - %s - (Hover for breakdown)' % ny_times_covid_date),
    html_filename = ('Covid19_cases_per_bed_per_county_%s.html' % time.strftime("%Y%m%d")),
    colorbar_title = '> Hazard Ratio (Cases/Bed)'
    )

### Identify counties with No PPE requests, with highest Covid19 cases

In [ ]:
covid_ppe_df = find_counties_with_covid19_and_no_ppe_request(covid_df, mask_df_counties)
covid_ppe_df[['date','county','state','cases','deaths','PPE_requests']].head(10)

### Correlation of PPE request per county with COVID19 cases

In [ ]:
# select counties that have had at least 1 ppe request
counties_with_ppe_requests_and_covid_cases = merged_covid_ppe_hosp_df[merged_covid_ppe_hosp_df.PPE_requests != 0]

# sort by highest normalized_covid_patients_per_bed
counties_with_ppe_requests_and_covid_cases.sort_values(by=['PPE_requests','cases'], ascending=False, inplace=True)
counties_with_ppe_requests_and_covid_cases.head(5)

In [ ]:
fig = px.scatter(
    counties_with_ppe_requests_and_covid_cases,
    x=counties_with_ppe_requests_and_covid_cases.cases, 
    y=counties_with_ppe_requests_and_covid_cases.PPE_requests,
    color='Covid_cases_per_bed',
    log_x=True,
    #log_y=True,
    labels={
        'Covid_cases_per_bed':'Covid19 cases per hospital bed',
        'x':'Covid19 Cases Per County',
        'y':'PPE Requests Per County',
        'text':'County'
        },
    hover_name=counties_with_ppe_requests_and_covid_cases.county,
    range_color=(0,1),
    range_x=(1,30000)
    )

fig.update_layout(
    title = "Correlation of PPE request per county with COVID19 cases",
    #hoverlabel={'text'},
    )

#fig.update_xaxes(nticks=30)
#fig.update_yaxes(nticks=20)
    
fig.show()

### Match hospital names to hospital database

In [ ]:
# TODO
''' 
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 

# this takes a while, so I limited it to 100 names
returned_matches = []
for name in mrl['name'].values.tolist()[:99]:
    best_matches = process.extract(name, std_hospital_names)
    returned_matches.append((name, best_matches))
'''
